In [ ]:
import uproot
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path

In [ ]:
def get_score_by_percentile(percentile, arr, weight=None):
    """
    Calculate score threshold for a given percentile.
    
    Args:
        percentile (float): Percentile value (0-1)
        arr (numpy.ndarray): Array of scores
        weight (numpy.ndarray, optional): Array of weights for weighted percentile
        
    Returns:
        float: Score threshold at the specified percentile
    """
    if weight is None:
        return np.percentile(arr, percentile * 100)
    else:
        # Sort both arrays based on scores
        sorted_indices = np.argsort(arr)
        sorted_arr = arr[sorted_indices]
        sorted_weights = weight[sorted_indices]
        
        # Calculate cumulative weights
        cumsum_weights = np.cumsum(sorted_weights)
        
        # Normalize weights to get percentiles (0-1)
        percentile_values = (cumsum_weights - sorted_weights/2) / cumsum_weights[-1]
        
        # Find the closest score to the desired percentile
        try:
            idx = np.searchsorted(percentile_values, percentile)
            return sorted_arr[idx]
        except IndexError:
            return sorted_arr[-1]

def get_percentile_by_score(score, arr, weight=None):
    """
    Calculate percentile rank for a given score.
    
    Args:
        score (float): Score threshold
        arr (numpy.ndarray): Array of scores
        weight (numpy.ndarray, optional): Array of weights
        
    Returns:
        float: Percentile rank (0-1)
    """
    if weight is None:
        # Count values less than or equal to score
        n_less = np.sum(arr <= score)
        return n_less / len(arr)
    else:
        # Get mask for values less than or equal to score
        mask = arr <= score
        
        # Sum weights for values less than or equal to score
        weight_less = np.sum(weight[mask])
        total_weight = np.sum(weight)
        
        return weight_less / total_weight

In [ ]:
TXbb_scores = [0.8, 0.9, 0.94, 0.97, 0.99, 1.0]
QCD_scores = {TXbb: [] for TXbb in TXbb_scores}

eras = ["2022", "2022EE", "2023BPix", "2023"]
DATA_DIR = Path("/afs/cern.ch/user/z/zichun/eos/higgs/HH4b-calib/HRT_ParT")
TXbb_all = []
weight_all = []
P_QCD_all = []
weight_TXbb_all = []

for era in eras:
    print(f"Processing era {era}")
    path = DATA_DIR / f"GloParTStage2_20250131_ak8_qcd_{era}/mc/outputs_sf_weights/ggfhh4b.root"
    events = uproot.open(path)["Events"]
    # genWeight*xsecWeight*puWeight
    genWeight = events["genWeight"].array(library="np")
    xsecWeight = events["xsecWeight"].array(library="np")
    puWeight = events["puWeight"].array(library="np")
    weight = genWeight*xsecWeight*puWeight

    TXbb_1 = events["fj_1_globalParT_XbbVsQCD"].array(library="np")
    TXbb_2 = events["fj_2_globalParT_XbbVsQCD"].array(library="np")

    TXbb_all.append(np.concatenate((TXbb_1, TXbb_2)))
    weight_all.append(np.concatenate((weight, weight)))
        
    # QCD
    path = DATA_DIR / f"GloParTStage2_20250131_ak8_qcd_{era}/mc/outputs_sf_weights/qcd.root"
    events = uproot.open(path)["Events"]
    genWeight = events["genWeight"].array(library="np")
    xsecWeight = events["xsecWeight"].array(library="np")
    puWeight = events["puWeight"].array(library="np")
    weight = genWeight*xsecWeight*puWeight
    P_QCD_1 = events["fj_1_globalParT_QCDb"].array(library="np")
    P_QCD_2 = events["fj_2_globalParT_QCDb"].array(library="np")

    SF_TXbb = events["SF_TXbb"].array(library="np")
    weight_TXbb = weight * SF_TXbb

    P_QCD_all.append(np.concatenate((P_QCD_1, P_QCD_2)))
    weight_TXbb_all.append(np.concatenate((weight_TXbb, weight_TXbb)))
    
TXbb_all = np.concatenate(TXbb_all)
weight_all = np.concatenate(weight_all)
P_QCD_all = np.concatenate(P_QCD_all)
weight_TXbb_all = np.concatenate(weight_TXbb_all)


In [ ]:
percentiles = []
for TXbb_score in TXbb_scores:
    percentile = get_percentile_by_score(score=TXbb_score, arr=TXbb_all, weight=weight_all)
    percentiles.append(percentile)
    print(f"Percentile for TXbb={TXbb_score}: {percentile}")
    
for percentile, TXbb_score in zip(percentiles, TXbb_scores):
    threshold = get_score_by_percentile(percentile=percentile, arr=P_QCD_all, weight=weight_TXbb_all)
    print(f"Score threshold for percentile {percentile} (TXbb={TXbb_score}): {threshold}")
    QCD_scores[TXbb_score].append(threshold)